# Storing the attribute in the Descriptor

In [93]:
class IntegerValue:
    def __init__(self):
        self.data = {}

    def __set__(self, instance, value):
        self.data[instance] = value

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.data.get(instance)

In [94]:
IntegerValue()

In [95]:
class Point:
    x = IntegerValue()

In [96]:
p1 = Point()

In [97]:
id_p1 = id(p1)
print(id_p1)

2150223621392


In [98]:
import ctypes


def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [99]:
ref_count(id_p1)

1

In [100]:
p1.x = 100

In [101]:
ref_count(id_p1)

2

In [102]:
del p1

In [103]:
ref_count(id_p1)

1

In [104]:
list(Point.x.data)[0]

In [105]:
hex(id_p1)

'0x1f4a350ad10'

this cause the memory leak.

To avoid the memory leak we can use hte WeakRefDict, which automatically delete the strong reference is dead

In [106]:
import weakref


class IntegerValue:
    def __init__(self):
        self.data = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.data[instance] = value

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.data.get(instance)

In [107]:
IntegerValue()

And that's all there is to it. We now have weak references instead of strong references in our dictionary, and the dictionary cleans up after itself (removes "dead" entries) when the reference object has been destroyed by the GC.

In [108]:
class Point:
    x = IntegerValue()

In [109]:
p1 = Point()
id_p1 = id(p1)

In [110]:
print(hex(id_p1))

0x1f4a34c1410


In [111]:
ref_count(id_p1)

1

In [112]:
p1.x = 100

In [113]:
ref_count(id_p1)

1

In [114]:
print(p1.__weakref__)

<weakref at 0x000001F4A34D3B50; to 'Point' at 0x000001F4A34C1410>


In [115]:
print(Point.x.data.keyrefs())

[<weakref at 0x000001F4A34D3B50; to 'Point' at 0x000001F4A34C1410>]


In [116]:
del p1

In [117]:
ref_count(id_p1)

-1673481500

In [118]:
Point.x.data.keyrefs()

[]

And if we delete p, thereby deleting the last strong reference to that object:

So this is almost a perfect general solution:

1. We do not need to store the data in the instances themselves (so we can handle objects whose class uses `__slots__`)
2. We are protected from memory leaks
But this only works for hashable objects.

## Hash-ability issue

Since we cannot use the object itself as the key in a dictionary (weak or otherwise), we could try using the id of the object (which is an int) as the key in a standard dictionary:

In [119]:
class IntegerValue:
    def __init__(self):
        self.data = {}

    def __set__(self, instance, value):
        self.data[id(instance)] = value

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.data.get(id(instance))

In [120]:
class Point:
    x = IntegerValue()

In [121]:
p1 = Point()

In [122]:
id_p1 = id(p1)

In [123]:
ref_count(id_p1)

1

In [124]:
p1.x = 100

In [125]:
ref_count(id_p1)

1

so far we didn't create memory leak

In [126]:
id_p1, Point.x.data

(2150223613776, {2150223613776: 100})

In [127]:
del p1

In [128]:
ref_count(id_p1)

-1640385308

In [129]:
Point.x.data

{2150223613776: 100}

But, we now have a "dead" entry in our dictionary - that memory address is still present as a key. Now, you might think it's not a big deal, but Python does reuse memory addresses, so we could run into potential issues there (where the data descriptor would have a value for a property already set from a previous object), and also the fact that our dictionary is cluttered with these dead entries:

### Solving the dea entry issue

In [130]:
p1 = Point()
id_p1 = id(p1)

In [131]:
ref_count(id_p1)

1

In [132]:
w1 = weakref.ref(p1)

In [133]:
w1

<weakref at 0x000001F4A3573C40; to 'Point' at 0x000001F4A3413250>

In [134]:
del p1

In [135]:
w1

<weakref at 0x000001F4A3573C40; dead>

1. **How the weakref is dead now?**
2. **How python we delete the strong reference?**

You can see that the weak reference was made aware of that change - in fact we can as well, by specifying a **callback** function that Python will call once the weak reference becomes dead (i.e. the object was destroyed by the GC):

In [136]:
def obj_destroyed(obj):
    print(type(obj))
    print(f"{obj} is destroyed")

In [137]:
p1 = Point()
w1 = weakref.ref(p1, obj_destroyed)

In [138]:
del p1

<class 'weakref.ReferenceType'>
<weakref at 0x000001F4A3572C00; dead> is destroyed


As you can see the callback function receives the weak ref object as the argument.

So, we can use this to our advantage in our data descriptor, by registering a callback that we can use to remove the "dead" entry from our values dictionary.

This means we do need to store a weak reference to the object as well - we'll do that in the value of the values dictionary as part of a tuple containing a weak reference to the object, and the corresponding value):



In [139]:
IntegerValue()

In [140]:
class IntegerValue:
    def __init__(self):
        self.data = {}

    def __set__(self, instance, value):
        self.data[id(instance)] = (weakref.ref(instance, self._finalizer), value)

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.data.get(id(instance))[1]

    def _finalizer(self, weak_ref):
        print("deleting the dead object in dict")
        for key, value in self.data.items():
            if value[0] is weak_ref:
                del self.data[key]
                break

In [141]:
class Point:
    x = IntegerValue()

In [142]:
p1 = Point()

In [143]:
p1.x = 100

In [144]:
id_p1 = id(p1)

In [145]:
ref_count(id_p1)

1

In [146]:
Point.x.data

{2150205606544: (<weakref at 0x000001F4A18798F0; to 'Point' at 0x000001F4A23DCA90>,
  100)}

In [147]:
del p1

deleting the dead object in dict


In [148]:
Point.x.data

{}

And as you can see our dictionary was cleaned up.

There is one last caveat, when we create weak references to objects, the weak reference objects are actually stored in the instance itself, in a property called __weakref__:

In [149]:
class Person:
    pass

In [150]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

Notice that `__weakref__` attribute. It is technically a data descriptor:


Now the problem if we use slots, is that the instances will no longer have that attribute!

In [151]:
class Person:
    __slots__ = "name",

In [152]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

So, the problem is that we can no longer create weak references to this object!!

In [153]:
try:
    weakref.ref(Person())
except TypeError as ex:
    print(ex)

cannot create weak reference to 'Person' object


In order to enable weak references in objects that use slots, we need to specify __weakref__ as one of the slots:



In [154]:
class Person:
    __slots__ = "name", "__weakref__"


In [155]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name', '__weakref__'),
              'name': <member 'name' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [156]:
try:
    weakref.ref(Person())
except TypeError as ex:
    print(ex)

# Final Approach

In [157]:
class ValidString:
    def __init__(self, min_length, max_length):
        self.data = {}
        self.min_length = min_length
        self.max_length = max_length

    def __set__(self, instance, value):
        if not isinstance(value,str):
            raise ValueError("Value must be string type")
        elif len(value) < self.min_length:
            raise ValueError(f"Value should have at least {self.min_length} character.")
        elif len(value) > self.max_length:
            raise ValueError(f"Value cannot exceed {self.max_length} character.")
        self.data[id(instance)] = (weakref.ref(instance,self._finalizer),value)

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.data.get(id(instance))[1]

    def _finalizer(self,weak_ref):
        reverse_look = [key for key,value in self.data.items()
                        if value[0] is weak_ref]
        if reverse_look:
            del self.data[reverse_look[0]]


In [158]:
class Person:
    __slots__ = "__weakref__"

    first_name = ValidString(1,10)
    last_name = ValidString(1,10)

    def __eq__(self, other):
        return (isinstance(other,Person) and
                self.first_name == other.first_name and
                self.last_name == other.last_name)

In [159]:
class BankAccount:
    __slots__ = "__weakref__"

    account_number = ValidString(1,6)

    def __eq__(self, other):
        return (isinstance(other,BankAccount) and
                self.account_number == other.account_number)

In [160]:
p1 = Person()

In [161]:
try:
    p1.first_name =""
except ValueError as ex:
    print(ex)

Value should have at least 1 character.


In [162]:
try:
    p1.last_name ="guhisdfgsdgjklsdghjsdsdlkh"
except ValueError as ex:
    print(ex)

Value cannot exceed 10 character.


In [163]:
b1 = BankAccount()

In [164]:
try:
    b1.account_number ="guhisdfgsdgjklsdghjsdsdlkh"
except ValueError as ex:
    print(ex)

Value cannot exceed 6 character.


In [165]:
p1 = Person()
p2 = Person()

In [166]:
b1 = BankAccount()
b2 = BankAccount()

In [167]:
p1.first_name = "first_p1"
p1.last_name = "last_p1"

In [168]:
p2.first_name = "first_p2"
p2.last_name = "last_p2"

In [169]:
p1.first_name,p1.last_name

('first_p1', 'last_p1')

In [170]:
p2.first_name,p2.last_name

('first_p2', 'last_p2')

In [171]:
b1.account_number="Save"
b2.account_number="Check"

In [172]:
b1.account_number,b2.account_number

('Save', 'Check')

In [173]:
Person.first_name.data

{2150215158752: (<weakref at 0x000001F4A2A7F420; to 'Person' at 0x000001F4A2CF8BE0>,
  'first_p1'),
 2150215157696: (<weakref at 0x000001F4A34F29D0; to 'Person' at 0x000001F4A2CF87C0>,
  'first_p2')}

In [174]:
Person.last_name.data

{2150215158752: (<weakref at 0x000001F4A358F240; to 'Person' at 0x000001F4A2CF8BE0>,
  'last_p1'),
 2150215157696: (<weakref at 0x000001F4A358F740; to 'Person' at 0x000001F4A2CF87C0>,
  'last_p2')}

In [175]:
BankAccount.account_number.data

{2150215158080: (<weakref at 0x000001F4A34B6930; to 'BankAccount' at 0x000001F4A2CF8940>,
  'Save'),
 2150215164080: (<weakref at 0x000001F49FF7B150; to 'BankAccount' at 0x000001F4A2CFA0B0>,
  'Check')}

In [176]:
del p1
del p2
del b1
del b2

In [177]:
Person.first_name.data,Person.last_name.data,BankAccount.account_number.data

({}, {}, {})